In [98]:
import numpy as np
import os
# !pip install tensorflow
from keras import layers, Sequential, regularizers
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_svmlight_file

In [30]:
def loadData( filename, dictSize = 225 ):
	X, y = load_svmlight_file( filename, multilabel = False, n_features = dictSize, offset = 1 )
	return (X, y)

X,y = loadData("train")
X = X.toarray()
y = y - 1
X.shape, y.shape
    

((10000, 225), (10000,))

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)
X_train.shape, y_test.shape

((8000, 225), (2000,))

In [132]:
model = Sequential([
    layers.Dense(128, activation='relu', input_shape=(225,)),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
    # layers.Dropout(0.2),
    # layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(1e-5)),
    layers.Dense(50, activation='softmax')
])

In [133]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [134]:
model.fit(X_train,y_train,epochs=500, batch_size=512, validation_data=(X_test, y_test))

Epoch 1/500
16/16 [==============================] - 1s 17ms/step - loss: 3.5469 - accuracy: 0.1870 - val_loss: 3.0457 - val_accuracy: 0.2635
Epoch 2/500
16/16 [==============================] - 0s 6ms/step - loss: 2.7965 - accuracy: 0.2996 - val_loss: 2.5744 - val_accuracy: 0.3920
Epoch 3/500
16/16 [==============================] - 0s 8ms/step - loss: 2.4864 - accuracy: 0.3815 - val_loss: 2.2909 - val_accuracy: 0.4545
Epoch 4/500
16/16 [==============================] - 0s 7ms/step - loss: 2.2435 - accuracy: 0.4498 - val_loss: 2.0418 - val_accuracy: 0.5130
Epoch 5/500
16/16 [==============================] - 0s 8ms/step - loss: 2.0383 - accuracy: 0.4979 - val_loss: 1.8323 - val_accuracy: 0.5690
Epoch 6/500
16/16 [==============================] - 0s 7ms/step - loss: 1.8666 - accuracy: 0.5280 - val_loss: 1.6786 - val_accuracy: 0.5980
Epoch 7/500
16/16 [==============================] - 0s 6ms/step - loss: 1.7225 - accuracy: 0.5644 - val_loss: 1.5478 - val_accuracy: 0.6165
Epoch 8/500


In [118]:
model.evaluate(X_test,y_test)

63/63 [==============================] - 0s 1ms/step - loss: 0.7254 - accuracy: 0.8390


[0.7253673672676086, 0.8389999866485596]

In [119]:
r = 5000
yp = model.predict(np.expand_dims(X[r],axis=0))
np.argmax(yp), y[r]

1/1 [==============================] - 0s 27ms/step


(8, 8.0)

In [128]:
from sample_submit.utils import getMPrecAtK, getPrecAtK

k = 5

y_test_prob = model.predict(X_test)
y_test_pred = np.argsort(y_test_prob, axis=1)[:,-k:] + 1
y_test_pred = np.flip(y_test_pred, axis=1)

y_test_50 = y_test + 1

getMPrecAtK(y_test_50, y_test_pred, k), getPrecAtK(y_test_50, y_test_pred, k)

63/63 [==============================] - 0s 1ms/step


(array([0.6144065 , 0.73051719, 0.84662136, 0.87934339, 0.89427209]),
 array([0.8335, 0.9215, 0.9525, 0.9655, 0.9755]))

In [121]:
model.save("submission/model.h5")

In [69]:
y_prob = model.predict(X)
y_pred = np.argsort(y_prob, axis=1)[:,-k:] + 1
ypred = np.flip(y_pred, axis=1)

y50 = y.astype(int) +1
getMPrecAtK(y50, ypred, k), getPrecAtK(y50, ypred, k)

313/313 [==============================] - 1s 2ms/step


(array([0.56398874, 0.69692462, 0.76931469, 0.82235712, 0.85918638]),
 array([0.7996, 0.8872, 0.9204, 0.9395, 0.952 ]))